In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix,roc_auc_score,roc_curve,auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import hamming_loss
inf552= pd.read_csv("C:\\Users\\DELL\\Desktop\\INF552\\HM5\\Frogs_MFCCs.csv")
#  Choose 70% of the data randomly as the training set, 30% as the test data
traindata=inf552.sample(frac=0.7,random_state=123 ,axis=0)
testdata=inf552.drop(index=traindata.index)

In [2]:
# divide traindata into X and Y
traindata_x=pd.DataFrame(traindata.iloc[:,:-4])
traindata_family=pd.DataFrame(traindata.iloc[:,-4])
traindata_genus=pd.DataFrame(traindata.iloc[:,-3])
traindata_species=pd.DataFrame(traindata.iloc[:,-2])
# divide testdata into X and Y
testdata_x=pd.DataFrame(testdata.iloc[:,:-4])
testdata_family=pd.DataFrame(testdata.iloc[:,-4])
testdata_genus=pd.DataFrame(testdata.iloc[:,-3])
testdata_species=pd.DataFrame(testdata.iloc[:,-2])

In [17]:
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import warnings
warnings.filterwarnings("ignore")

In [140]:
#determine the ranges for σ and λ that keep the accuracy above a threshold (e.g.60%)
cls_family = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,C=10**(-2))).fit(traindata_x, traindata_family)
print('the accuracy for λ = 10−2 is: ',cls_family.score(traindata_x, traindata_family))
cls_family2 = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,C=10**(5))).fit(traindata_x, traindata_family)
print('the accuracy for λ = 10+5 is: ',cls_family2.score(traindata_x, traindata_family))
print('the accuracy will not below 60% for λ = 10−2 and λ = 10+5')
cls_family3 = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,gamma=0.0001)).fit(traindata_x, traindata_family)
print('the accuracy for gamma = 10−4 is: ',cls_family3.score(traindata_x, traindata_family))
cls_family4 = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,gamma=100)).fit(traindata_x, traindata_family)
print('the accuracy for gamma = 10+2 is: ',cls_family4.score(traindata_x, traindata_family))
print('the accuracy will not below 60% for gamma = 10−4 and gamma = 10+2')

the accuracy for λ = 10−2 is:  0.607823669579031
the accuracy for λ = 10+5 is:  1.0
the accuracy will not below 60% for λ = 10−2 and λ = 10+5
the accuracy for gamma = 10−4 is:  0.607823669579031
the accuracy for gamma = 10+2 is:  1.0
the accuracy will not below 60% for gamma = 10−4 and gamma = 10+2


In [142]:
#Train a SVM for each of the family labels, using Gaussian kernels and one versus all classiﬁers. 
# select a fixed number of parameters (e.g.,10) in the parameters' range  and use them for 10 fold cross validation. 
C_range = np.logspace(-2, 5, 10)
gamma_range = np.linspace(0.0001, 100,10)
param_grid = dict(estimator__degree=gamma_range,estimator__C=C_range)
cls = OneVsRestClassifier(SVC(kernel='rbf',random_state=123))
model_family =  GridSearchCV(cls, param_grid=param_grid, cv=10).fit(traindata_x, traindata_family)

In [143]:
# evaluate the classiﬁers for family using Exact match and Hamming loss 
test_pred=model_family.predict(testdata_x)
f1=accuracy_score(testdata_family,test_pred);f2=hamming_loss(testdata_family,test_pred);
print('Exact match score: ',f1)
print('Hamming loss score: ',f2)

Exact match score:  0.9874942102825383
Hamming loss score:  0.012505789717461788


In [149]:
#determine the ranges for σ and λ that keep the accuracy above a threshold (e.g.60%)
cls_genus = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,C=10**(-1.5))).fit(traindata_x, traindata_genus)
print('the accuracy for λ = 10−1.5 is: ',cls_genus.score(traindata_x, traindata_genus))
cls_genus2 = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,C=10**(5))).fit(traindata_x, traindata_genus)
print('the accuracy for λ = 10+5 is: ',cls_genus2.score(traindata_x, traindata_genus))
print('the accuracy will not below 60% for λ = 10−1.5 and λ = 10+5')
cls_genus3 = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,gamma=10**(-3))).fit(traindata_x, traindata_genus)
print('the accuracy for gamma = 10−3 is: ',cls_genus3.score(traindata_x, traindata_genus))
cls_genus4 = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,gamma=10**(1))).fit(traindata_x, traindata_genus)
print('the accuracy for gamma = 10+1 is: ',cls_genus4.score(traindata_x, traindata_genus))
print('the accuracy will not below 60% for gamma = 10−3 and gamma = 10+1')

the accuracy for λ = 10−1.5 is:  0.6479348689436061
the accuracy for λ = 10+5 is:  1.0
the accuracy will not below 60% for λ = 10−1.5 and λ = 10+5
the accuracy for gamma = 10−3 is:  0.6336378077839555
the accuracy for gamma = 10+1 is:  1.0
the accuracy will not below 60% for gamma = 10−3 and gamma = 10+1


In [150]:
#Train a SVM for the genus labels
C_range = np.logspace(-1.5, 5, 10)
gamma_range = np.linspace(0.001, 10,10)
param_grid = dict(estimator__degree=gamma_range,estimator__C=C_range)
model_genus = GridSearchCV(cls, param_grid=param_grid, cv=10).fit(traindata_x, traindata_genus)

In [151]:
# evaluate the classiﬁers using Exact match and Hamming loss 
test_pred2=model_genus.predict(testdata_x)
print('Classifer for Genus')
g1=accuracy_score(testdata_genus,test_pred2);g2=hamming_loss(testdata_genus,test_pred2);
print('Exact match score: ',g1)
print('Hamming loss score: ',g2)

Classifer for Genus
Exact match score:  0.9879573876794813
Hamming loss score:  0.012042612320518759


In [160]:
#determine the ranges for σ and λ that keep the accuracy above a threshold (e.g.60%)
cls_species = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,C=10**(-1.7))).fit(traindata_x, traindata_species)
print('the accuracy for λ = 10−1.7 is: ',cls_species.score(traindata_x, traindata_species))
cls_species2 = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,C=10**(5))).fit(traindata_x, traindata_species)
print('the accuracy for λ = 10+5 is: ',cls_species2.score(traindata_x, traindata_species))
print('the accuracy will not below 60% for λ = 10−1.7 and λ = 10+5')
cls_species3 = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,gamma=10**(-3))).fit(traindata_x, traindata_species)
print('the accuracy for gamma = 10−3 is: ',cls_species3.score(traindata_x, traindata_species))
cls_species4 = OneVsRestClassifier(SVC(kernel='rbf',random_state=123,gamma=10**(1))).fit(traindata_x, traindata_species)
print('the accuracy for gamma = 10+1 is: ',cls_species4.score(traindata_x, traindata_species))
print('the accuracy will not below 60% for gamma = 10−3 and gamma = 10+1')

the accuracy for λ = 10−1.7 is:  0.6266878474980143
the accuracy for λ = 10+5 is:  1.0
the accuracy will not below 60% for λ = 10−1.7 and λ = 10+5
the accuracy for gamma = 10−3 is:  0.6384034948371724
the accuracy for gamma = 10+1 is:  1.0
the accuracy will not below 60% for gamma = 10−3 and gamma = 10+1


In [158]:
#Train a SVM for the species labels
C_range = np.logspace(-1.7, 5, 10)
gamma_range = np.linspace(0.0001, 10,10)
param_grid = dict(estimator__degree=gamma_range,estimator__C=C_range)
model_species =  GridSearchCV(cls, param_grid=param_grid, cv=10).fit(traindata_x, traindata_species)

In [159]:
# evaluate the classiﬁers using Exact match and Hamming loss 
test_pred3=model_species.predict(testdata_x)
s1=accuracy_score(testdata_species,test_pred3);s2=hamming_loss(testdata_species,test_pred3);
print('Exact match score: ',s1)
print('Hamming loss score: ',s2)

Exact match score:  0.9874942102825383
Hamming loss score:  0.012505789717461788


In [197]:
from sklearn import preprocessing
from sklearn.svm import LinearSVC
train_x=preprocessing.scale(traindata_x)
#determine the ranges for λ that keep the accuracy above a threshold (e.g.60%)
cls2_family = OneVsRestClassifier(LinearSVC(penalty='l1',random_state=123,dual=False,C=10**(-5))).fit(train_x, traindata_family)
print('the accuracy for λ = 10−5 is: ',cls2_family.score(train_x, traindata_family))
cls2_family2 = OneVsRestClassifier(LinearSVC(penalty='l1',random_state=123,dual=False,C=10**(1))).fit(train_x, traindata_family)
print('the accuracy for λ = 10+1 is: ',cls2_family2.score(train_x, traindata_family))
print('the accuracy will not below 60% for λ = 10−5 and λ = 10+1')

the accuracy for λ = 10−5 is:  0.607823669579031
the accuracy for λ = 10+1 is:  0.9426131850675139
the accuracy will not below 60% for λ = 10−5 and λ = 10+1


In [183]:
# Train the L1-penalized SVMs for family using standardized attributes
# Determine the weight of the SVM penalty using 10 fold cross validation. 
from sklearn import preprocessing
from sklearn.svm import LinearSVC
train_x=preprocessing.scale(traindata_x)
C_range2 = np.logspace(-5, 1, 10)
param_grid2 = dict(estimator__C=C_range)
cls2 = OneVsRestClassifier(LinearSVC(penalty='l1',random_state=123,dual=False))
model2_family =  GridSearchCV(cls2, param_grid=param_grid2, cv=10).fit(train_x, traindata_family)

In [184]:
# evaluate the classiﬁers for family using Exact match and Hamming loss
test_x=preprocessing.scale(testdata_x)
pred=model2_family.predict(test_x)
fa1=accuracy_score(testdata_family,pred);fa2=hamming_loss(testdata_family,pred);
print('Exact match score: ',fa1)
print('Hamming loss score: ',fa2)

Exact match score:  0.9282075034738305
Hamming loss score:  0.07179249652616952


In [198]:
#determine the ranges for λ that keep the accuracy above a threshold (e.g.60%)
cls2_genus = OneVsRestClassifier(LinearSVC(penalty='l1',random_state=123,dual=False,C=10**(-3.8))).fit(train_x, traindata_genus)
print('the accuracy for λ = 10−3.8 is: ',cls2_genus.score(train_x, traindata_genus))
cls2_genus2 = OneVsRestClassifier(LinearSVC(penalty='l1',random_state=123,dual=False,C=10**(4))).fit(train_x, traindata_genus)
print('the accuracy for λ = 10+4 is: ',cls2_genus2.score(train_x, traindata_genus))
print('the accuracy will not below 60% for λ = 10−3.8 and λ = 10+4')

the accuracy for λ = 10−3.8 is:  0.687251787132645
the accuracy for λ = 10+4 is:  0.9575059571088165
the accuracy will not below 60% for λ = 10−3.8 and λ = 10+4


In [199]:
# Train the L1-penalized SVMs for the genus labels using standardized attributes
C_range2 = np.logspace(-3.8, 4, 10)
param_grid2 = dict(estimator__C=C_range)
model2_genus =  GridSearchCV(cls2, param_grid=param_grid2, cv=10).fit(train_x, traindata_genus)

In [200]:
# evaluate the classiﬁers using Exact match and Hamming loss 
pred2=model2_genus.predict(test_x)
ge1=accuracy_score(testdata_genus,pred2);ge2= hamming_loss(testdata_genus,pred2);
print('Classifer for Genus')
print('Exact match score: ',ge1)
print('Hamming loss score: ',ge2)

Classifer for Genus
Exact match score:  0.9550717924965262
Hamming loss score:  0.04492820750347383


In [209]:
#determine the ranges for λ that keep the accuracy above a threshold (e.g.60%)
cls2_species = OneVsRestClassifier(LinearSVC(penalty='l1',random_state=123,dual=False,C=10**(-3.4))).fit(train_x, traindata_species)
print('the accuracy for λ = 10−3.4 is: ',cls2_species.score(train_x, traindata_species))
cls2_species2 = OneVsRestClassifier(LinearSVC(penalty='l1',random_state=123,dual=False,C=10**(4))).fit(train_x, traindata_species)
print('the accuracy for λ = 10+4 is: ',cls2_species2.score(train_x, traindata_species))
print('the accuracy will not below 60% for λ = 10−3.4 and λ = 10+4')

the accuracy for λ = 10−3.4 is:  0.6123907863383637
the accuracy for λ = 10+4 is:  0.9646544876886418
the accuracy will not below 60% for λ = 10−3.4 and λ = 10+4


In [210]:
# Train the L1-penalized SVMs for the spcies labels using standardized attributes
C_range2 = np.logspace(-3.4, 4, 10)
param_grid2 = dict(estimator__C=C_range)
model2_species =  GridSearchCV(cls2, param_grid=param_grid2, cv=10).fit(train_x, traindata_species)

In [211]:
# evaluate the classiﬁers using Exact match and Hamming loss 
pred3=model2_species.predict(test_x)
sp1=accuracy_score(testdata_species,pred3);sp2=hamming_loss(testdata_species,pred3);
print('Classifer for Species')
print('Exact match score: ',sp1)
print('Hamming loss score: ',sp2)

Classifer for Species
Exact match score:  0.9587772116720704
Hamming loss score:  0.041222788327929596


In [212]:
# using SMOTE to remedy class imbalance
import imblearn
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=123)
os_data_x,os_data_family=os.fit_sample(train_x, traindata_family)
os_data_x=pd.DataFrame(os_data_x)
os_data_family=pd.DataFrame(os_data_family)
os_data_x2,os_data_genus=os.fit_sample(train_x, traindata_genus)
os_data_x2=pd.DataFrame(os_data_x2)
os_data_genus=pd.DataFrame(os_data_genus)
os_data_x3,os_data_species=os.fit_sample(train_x, traindata_species)
os_data_x3=pd.DataFrame(os_data_x3)
os_data_species=pd.DataFrame(os_data_species)
C_range31 = np.logspace(-5, 1, 10)
param_grid31 = dict(estimator__C=C_range31)
model3_family =  GridSearchCV(cls2, param_grid=param_grid31, cv=10).fit(os_data_x, os_data_family)
C_range32 = np.logspace(-3.8, 4, 10)
param_grid32 = dict(estimator__C=C_range32)
model3_genus =  GridSearchCV(cls2, param_grid=param_grid32, cv=10).fit(os_data_x2, os_data_genus)
C_range33 = np.logspace(-3.4, 4, 10)
param_grid33 = dict(estimator__C=C_range33)
model3_species =  GridSearchCV(cls2, param_grid=param_grid33, cv=10).fit(os_data_x3, os_data_species)

In [213]:
# evaluate the classiﬁers using Exact match and Hamming loss 
prediction=model3_family.predict(test_x)
prediction2=model3_genus.predict(test_x)
prediction3=model3_species.predict(test_x)
print('Classifer for Family')
family1=accuracy_score(testdata_family,prediction);family2=hamming_loss(testdata_family,prediction);
print('Exact match score: ',family1)
print('Hamming loss score: ',family2)
print('Classifer for Genus')
genus1=accuracy_score(testdata_genus,prediction2);genus2=hamming_loss(testdata_genus,prediction2);
print('Exact match score: ',genus1)
print('Hamming loss score: ',genus2)
print('Classifer for Species')
species1=accuracy_score(testdata_species,prediction3);species2=hamming_loss(testdata_species,prediction3);
print('Exact match score: ',species1)
print('Hamming loss score: ',species2)

Classifer for Family
Exact match score:  0.9064381658175081
Hamming loss score:  0.0935618341824919
Classifer for Genus
Exact match score:  0.904122278832793
Hamming loss score:  0.09587772116720704
Classifer for Species
Exact match score:  0.953682260305697
Hamming loss score:  0.04631773969430292


In [214]:
# conclusions about the classiﬁers
from prettytable import PrettyTable 
table=PrettyTable([" Exact match ratio"," SVM(Gaussian kernels)"," L1-penalized SVM"," L1-penalized SVM (SMOTE)"])
table.add_row(["family",f1,fa1,family1])
table.add_row(["genus",g1,ge1,genus1])
table.add_row(["species",s1,sp1,species1])
table2=PrettyTable([" Hamming score"," SVM(Gaussian kernels)"," L1-penalized SVM"," L1-penalized SVM (SMOTE)"])
table2.add_row(["family",f2,fa2,family2])
table2.add_row(["genus",g2,ge2,genus2])
table2.add_row(["species",s2,sp2,species2])
print(table);
print('for exact match ration, SVM(Gaussian kernels) > L1-penalized SVM > L1-penalized SVM (SMOTE)')
print(table2);
print('for exact hamming score, SVM(Gaussian kernels) < L1-penalized SVM < L1-penalized SVM (SMOTE)')

+--------------------+------------------------+--------------------+---------------------------+
|  Exact match ratio |  SVM(Gaussian kernels) |  L1-penalized SVM  |  L1-penalized SVM (SMOTE) |
+--------------------+------------------------+--------------------+---------------------------+
|       family       |   0.9874942102825383   | 0.9282075034738305 |     0.9064381658175081    |
|       genus        |   0.9879573876794813   | 0.9550717924965262 |     0.904122278832793     |
|      species       |   0.9874942102825383   | 0.9587772116720704 |     0.953682260305697     |
+--------------------+------------------------+--------------------+---------------------------+
for exact match ration, SVM(Gaussian kernels) > L1-penalized SVM > L1-penalized SVM (SMOTE)
+----------------+------------------------+----------------------+---------------------------+
|  Hamming score |  SVM(Gaussian kernels) |   L1-penalized SVM   |  L1-penalized SVM (SMOTE) |
+----------------+---------------------

 Extra Practice: Study the Classiﬁer Chain method and apply it to the above problem.

In [30]:
from sklearn.multioutput import ClassifierChain
X_train=pd.DataFrame(traindata.iloc[:,:-4])
Y_train=pd.DataFrame(traindata.iloc[:,-4:-1])
Y_train['Family'].replace('Bufonidae',1,inplace=True);Y_train['Family'].replace('Dendrobatidae',2,inplace=True);Y_train['Family'].replace('Hylidae',3,inplace=True);Y_train['Family'].replace('Leptodactylidae',4,inplace=True);
Y_train['Genus'].replace('Adenomera',1,inplace=True);Y_train['Genus'].replace('Ameerega',2,inplace=True);Y_train['Genus'].replace('Dendropsophus',3,inplace=True);Y_train['Genus'].replace('Hypsiboas',4,inplace=True);
Y_train['Genus'].replace('Leptodactylus',5,inplace=True);Y_train['Genus'].replace('Osteocephalus',6,inplace=True);Y_train['Genus'].replace('Rhinella',7,inplace=True);Y_train['Genus'].replace('Scinax',8,inplace=True);
Y_train['Species'].replace('AdenomeraAndre',1,inplace=True);Y_train['Species'].replace('AdenomeraHylaedactylus',2,inplace=True);Y_train['Species'].replace('Ameeregatrivittata',3,inplace=True);Y_train['Species'].replace('HylaMinuta',4,inplace=True);
Y_train['Species'].replace('HypsiboasCinerascens',5,inplace=True);Y_train['Species'].replace('HypsiboasCordobae',6,inplace=True);Y_train['Species'].replace('LeptodactylusFuscus',7,inplace=True);Y_train['Species'].replace('OsteocephalusOophagus',8,inplace=True);
Y_train['Species'].replace('Rhinellagranulosa',9,inplace=True);Y_train['Species'].replace('ScinaxRuber',10,inplace=True);
X_test=pd.DataFrame(testdata.iloc[:,:-4])
Y_test=pd.DataFrame(testdata.iloc[:,-4:-1])
Y_test['Family'].replace('Bufonidae',1,inplace=True);Y_test['Family'].replace('Dendrobatidae',2,inplace=True);Y_test['Family'].replace('Hylidae',3,inplace=True);Y_test['Family'].replace('Leptodactylidae',4,inplace=True);
Y_test['Genus'].replace('Adenomera',1,inplace=True);Y_test['Genus'].replace('Ameerega',2,inplace=True);Y_test['Genus'].replace('Dendropsophus',3,inplace=True);Y_test['Genus'].replace('Hypsiboas',4,inplace=True);
Y_test['Genus'].replace('Leptodactylus',5,inplace=True);Y_test['Genus'].replace('Osteocephalus',6,inplace=True);Y_test['Genus'].replace('Rhinella',7,inplace=True);Y_test['Genus'].replace('Scinax',8,inplace=True);
Y_test['Species'].replace('AdenomeraAndre',1,inplace=True);Y_test['Species'].replace('AdenomeraHylaedactylus',2,inplace=True);Y_test['Species'].replace('Ameeregatrivittata',3,inplace=True);Y_test['Species'].replace('HylaMinuta',4,inplace=True);
Y_test['Species'].replace('HypsiboasCinerascens',5,inplace=True);Y_test['Species'].replace('HypsiboasCordobae',6,inplace=True);Y_test['Species'].replace('LeptodactylusFuscus',7,inplace=True);Y_test['Species'].replace('OsteocephalusOophagus',8,inplace=True);
Y_test['Species'].replace('Rhinellagranulosa',9,inplace=True);Y_test['Species'].replace('ScinaxRuber',10,inplace=True);

In [31]:
C = np.logspace(-3, 3, 10)
gamma = np.logspace(-3, 1,10)
param = dict(gamma=gamma, C=C)
#svm= OneVsRestClassifier(SVC(kernel='rbf',random_state=123))
svm = GridSearchCV(SVC(kernel='rbf',random_state=123), param_grid=param, cv=3)
#baseline = GridSearchCV(svm, param_grid=param, cv=3)
chains = [ClassifierChain(svm, order='random', random_state=i) for i in range(3)]
j=0
for chain in chains:
    chain.fit(X_train, pd.DataFrame(Y_train.iloc[:,j]))
    j+=1
Y_pred_chains = np.array([chain.predict(X_test) for chain in chains])

In [32]:
# classifier chains
h=0
for Y_pred_chain in Y_pred_chains:
    print('classifier',(h+1),'exact match is ',accuracy_score(Y_test.iloc[:,h], Y_pred_chain))
    print('classifier',(h+1),'hamming score is ',hamming_loss(Y_test.iloc[:,h], Y_pred_chain))
    h+=1

classifier 1 exact match is  0.9916628068550255
classifier 1 hamming score is  0.008337193144974525
classifier 2 exact match is  0.9921259842519685
classifier 2 hamming score is  0.007874015748031496
classifier 3 exact match is  0.9898100972672533
classifier 3 hamming score is  0.010189902732746642


In [33]:
# K-means( Selecting K with CH index )
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabaz_score
data_x=pd.DataFrame(inf552.iloc[:,:-4])
#data_x=preprocessing.scale(data_x)
range_K=range(2,11)
for i in range_K:
    clusterer = KMeans(n_clusters=i,random_state=123)
    cluster_labels = clusterer.fit_predict(data_x)
    CH_avg = calinski_harabaz_score(data_x, cluster_labels)
    print("For n_clusters =", i,"The average CH is :", CH_avg)
print('choose the value of K=2 with the largest score CH(K).')

For n_clusters = 2 The average CH is : 3680.7413494630937
For n_clusters = 3 The average CH is : 3157.6375348307733
For n_clusters = 4 The average CH is : 2756.4749894406414
For n_clusters = 5 The average CH is : 2628.953027816781
For n_clusters = 6 The average CH is : 2390.2350026691993
For n_clusters = 7 The average CH is : 2235.9894596927315
For n_clusters = 8 The average CH is : 2128.497449664384
For n_clusters = 9 The average CH is : 2049.158324338749
For n_clusters = 10 The average CH is : 1974.2065139372091
choose the value of K=2 with the largest score CH(K).


In [34]:
#In each cluster, determine which class is the majority by reading the true labels
cluster = KMeans(n_clusters=2,random_state=123)
labels = cluster.fit_predict(data_x)
print('for cluster 0, the major family is Leptodactylidae.')
print('for cluster 1, the major family is Hylidae.')
pd.crosstab(inf552.iloc[:,-4],labels)

for cluster 0, the major family is Leptodactylidae.
for cluster 1, the major family is Hylidae.


col_0,0,1
Family,,
Bufonidae,0,68
Dendrobatidae,0,542
Hylidae,124,2041
Leptodactylidae,3474,946


In [35]:
print('for cluster 0, the major genus is Adenomera.')
print('for cluster 1, the major genus is Hypsiboas.')
pd.crosstab(inf552.iloc[:,-3],labels)

for cluster 0, the major genus is Adenomera.
for cluster 1, the major genus is Hypsiboas.


col_0,0,1
Genus,,
Adenomera,3470,680
Ameerega,0,542
Dendropsophus,108,202
Hypsiboas,14,1579
Leptodactylus,4,266
Osteocephalus,0,114
Rhinella,0,68
Scinax,2,146


In [36]:
print('for cluster 0, the major species is AdenomeraHylaedactylus.')
print('for cluster 1, the major species is HypsiboasCordobae.')
pd.crosstab(inf552.iloc[:,-2],labels)

for cluster 0, the major species is AdenomeraHylaedactylus.
for cluster 1, the major species is HypsiboasCordobae.


col_0,0,1
Species,,
AdenomeraAndre,0,672
AdenomeraHylaedactylus,3470,8
Ameeregatrivittata,0,542
HylaMinuta,108,202
HypsiboasCinerascens,0,472
HypsiboasCordobae,14,1107
LeptodactylusFuscus,4,266
OsteocephalusOophagus,0,114
Rhinellagranulosa,0,68


In [37]:
#Calculate the average Hamming distance (score) between the true labels and the labels assigned by clusters. 
multi=[]
for i in labels:
    if i==0:
        multi.append(['Leptodactylidae','Adenomera','AdenomeraHylaedactylus'])
    else:
        multi.append(['Hylidae','Hypsiboas','HypsiboasCordobae'])
data_y=pd.DataFrame(inf552.iloc[:,-4:-1])
hamming_score=0
for i in range(len(data_y)):
    hamming_score+=hamming_loss(data_y.values[i],multi[i])
print('the average Hamming distance (score): ',hamming_score/len(data_y))

the average Hamming distance (score):  0.29854065323140605


In [38]:
# Monte-Carlo Simulation: Perform the procedures above 50 times
hamming=[]
for n in range(50):
    ch_avg=[0]*10;best_k,maximum=1,0;
    for i in range(2,11):
        clu = KMeans(n_clusters=i)
        clu_labels = clu.fit_predict(data_x)
        ch_avg[i-1] = calinski_harabaz_score(data_x, clu_labels)
        if ch_avg[i-1]>maximum:
            best_k=i;maximum=ch_avg[i-1];          
    best_clu = KMeans(n_clusters=best_k)
    ls = best_clu.fit_predict(data_x)
    label_triplet=[] 
    for j in range(best_k):
        triplet=[]
        triplet.append(pd.crosstab(inf552.iloc[:,-4],ls)[j].idxmax())
        triplet.append(pd.crosstab(inf552.iloc[:,-3],ls)[j].idxmax())
        triplet.append(pd.crosstab(inf552.iloc[:,-2],ls)[j].idxmax())
        label_triplet.append(triplet)
    multiple=[]
    for v in ls:
        if v==0:
            multiple.append(label_triplet[0])
        else:
            multiple.append(label_triplet[1])
    ham_score=0
    for w in range(len(data_y)):
        ham_score+=hamming_loss(data_y.values[w],multiple[w])
    hamming.append(ham_score/len(data_y))

In [39]:
# report the average and standard deviation of the 50 Hamming Distances
print('the averge: ',np.mean(hamming))
print('the std:',np.std(hamming))

the averge:  0.29854065323140605
the std: 0.0
